In [16]:
#### MAY NEED TO REMOVE --> THESE IMPORTS ARE JUST FOR MY DEPENDENCIES ON MY LOCAL DEVICE
import sys 
sys.path.append('../..')
######

from cox.utils import Parameters
from cox.store import Store
from cox.readers import CollectionReader
import matplotlib.pyplot as plt
import seaborn as sns
import torch as ch
from torch import Tensor
from torch import sigmoid as sig
import torch.nn as nn
from torch.optim import SGD, lr_scheduler
from torch.distributions import Gumbel, Uniform
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.transforms import SigmoidTransform
from torch.distributions.transformed_distribution import TransformedDistribution
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from tqdm.autonotebook import tqdm as tqdm
from abc import ABC
import IPython
import os
import dill
import config

from delphi import train
from delphi import oracle
from delphi import grad
import delphi.utils.constants as consts
from delphi.utils.helpers import logistic
# set environment variable so that stores can create output files
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

In [50]:
config.args = Parameters({
    'in_features': 2, 
    'k': 2,
    'param_lower': -1, 
    'param_upper': 1, 
    'x_lower': -5, 
    'x_upper': 5,
    'samples': 10000,
    'num_samples': 1000,
})
config.args

{
  "in_features": 2,
  "k": 2,
  "param_lower": -1,
  "param_upper": 1,
  "x_lower": -5,
  "x_upper": 5,
  "samples": 10000,
  "num_samples": 1000
}

In [10]:
# oracles
id_ = oracle.Identity()

Distributions

In [17]:
# gumbel noise distribution
gumbel = Gumbel(0, 1)

Gradients

In [18]:
logistic_bce = grad.LogisticBCE.apply
from torch.nn import BCEWithLogitsLoss; bce_logit = BCEWithLogitsLoss()
trunc_ce = grad.TruncatedCE.apply
trunc_bce = grad.TruncatedBCE.apply
gumbel_ce = grad.GumbelCE.apply
from torch.nn import CrossEntropyLoss; ce_loss = CrossEntropyLoss()

In [92]:
class GumbelBCE(ch.autograd.Function):
    @staticmethod
    def forward(ctx, pred, targ):
        ctx.save_for_backward(pred, targ)
        loss = CrossEntropyLoss()
        return loss(pred, targ)

    @staticmethod
    def backward(ctx, grad_output):
        pred, targ = ctx.saved_tensors

        # Gumbel distribution
        gumbel = Gumbel(0, 1)

        stacked = pred[None, ...].repeat(config.args.num_samples, 1, 1)
        rand_noise = gumbel.sample(stacked.size())
        # add noise
        noised = stacked + rand_noise
        noised_labs = noised.argmax(-1)
        # filter
        mask = (noised_labs).eq(targ)
        diff_rand_noise = rand_noise[:,:,0] - rand_noise[:,:,1]
        avg = 1 - 2*((sig(diff_rand_noise)*mask).sum(0) / (mask.sum(0) + 1e-5))
        return ch.stack([avg, ch.zeros(avg.size())], dim=1), None

gumbel_bce = GumbelBCE.apply

In [65]:
def gradient_similarity(args, grad_1, grad_2, d_n):
    """
    grad_1: first gradient
    grad_2: second gradient 
    d_n: latent variable noise distribution
    returns: grad_1 and grad_2 gradients with respect to the noised output
    """
    # generate random uniform weights
    ground_truth = nn.Linear(in_features=args.in_features, out_features=args.k, bias=args.bias)
    ground_truth.weight = nn.Parameter(Uniform(args.param_lower, args.param_upper).sample(ch.Size([1, args.in_features])))
    if ground_truth.bias is not None: 
        ground_truth.bias = nn.Parameter(Uniform(args.param_lower, args.param_upper).sample(ch.Size([1,])))

    # generate data
    X = Uniform(-5, 5).sample(ch.Size([args.samples, args.in_features]))
    z = ground_truth(X) + d_n.sample(ch.Size([X.size(0), 1]))
    y = args.pi(z)

    # truncate
    indices = ch.all(args.phi(z).bool(), dim=1).nonzero(as_tuple=False).flatten()
    x_trunc = X[indices]
    y_trunc = y[indices]

    print("alpha: ", x_trunc.size(0) / X.size(0))

    # pass data through model
    model = ch.nn.Linear(in_features=args.in_features, out_features=args.k)
    pred = model(x_trunc)

    g1_loss = grad_1(pred, y_trunc)
    g1_g, = ch.autograd.grad(g1_loss, [pred])

    g2_loss = grad_2(pred, y_trunc)
    g2_g, = ch.autograd.grad(g2_loss, [pred])


    print('Cosine Similarity between grad 1 and grad 2: ',
            ch.nn.CosineSimilarity(dim=1)(g2_g.T, g1_g.T))
    
    return g1_g, g2_g

In [94]:
def gradient_similarity_gumbel_diff(args, gum_grad, grad_2, d_n):
    """
    gum_grad: gumbel gradient
    grad_2: second gradient 
    d_n: latent variable noise distribution
    returns: grad_1 and grad_2 gradients with respect to the noised output
    """
    # generate random uniform weights
    ground_truth = nn.Linear(in_features=args.in_features, out_features=args.k, bias=args.bias)
    ground_truth.weight = nn.Parameter(Uniform(args.param_lower, args.param_upper).sample(ch.Size([1, args.in_features])))
    if ground_truth.bias is not None: 
        ground_truth.bias = nn.Parameter(Uniform(args.param_lower, args.param_upper).sample(ch.Size([1,])))

    # generate data
    X = Uniform(-5, 5).sample(ch.Size([args.samples, args.in_features]))
    z = ground_truth(X) + d_n.sample(ch.Size([X.size(0), 1]))
    y = args.pi(z)

    # truncate
    indices = ch.all(args.phi(z).bool(), dim=1).nonzero(as_tuple=False).flatten()
    x_trunc = X[indices]
    y_trunc = y[indices]
    print("alpha: ", x_trunc.size(0) / X.size(0))

    # pass data through model
    
    model = ch.nn.Linear(in_features=args.in_features, out_features=args.k)
    pred = model(x_trunc)
    
    g1_loss = gum_grad(pred, y_trunc.long().flatten())
    g1_g, = ch.autograd.grad(g1_loss, [pred])
    
    g1_diff = g1_g[:,0] - g1_g[:, 1]
    g1 = -g1_g[:,0]

    gum_diff = pred[:,0] - pred[:,1]

    g2_loss = grad_2(pred, y_trunc)
    g2_g, = ch.autograd.grad(g2_loss, [pred])

    print('Cosine Similarity between grad 1 and grad 2: ',
            ch.nn.CosineSimilarity(dim=0)(g2_g.sum(1).unsqueeze(1), g1_g[:,1].unsqueeze(1)))
    
    print('Cosine Similarity between grad 1 and grad 2: ',
        ch.nn.CosineSimilarity(dim=0)(g2_g.sum(1).unsqueeze(1), g1_g[:,0].unsqueeze(1)))
    
    
    return g2_g.sum(1).mean(0).reshape(1, 1), g2_g 


config.args.__setattr__('pi', multi)
config.args.__setattr__('k', 2)
# config.args.__setattr__('phi', oracle.DNN_Logit_Ball(ch.full(ch.Size([config.args.K,]), -2, dtype=ch.float32), ch.full(ch.Size([config.args.K,]), 2, dtype=ch.float32)))
config.args.__setattr__('phi', id_)
g1, g2 = gradient_similarity_gumbel_diff(config.args, gumbel_ce, gumbel_bce, gumbel)

alpha:  1.0
Cosine Similarity between grad 1 and grad 2:  tensor([-0.9831])
Cosine Similarity between grad 1 and grad 2:  tensor([0.9994])


Projection Mechanisms

In [6]:
binary = lambda z: ch.where(z > 0, ch.ones(1), ch.zeros(1))
multi = lambda z: z.argmax(-1)

Compare CE Loss with Truncated CE Loss with Identity Oracle

In [11]:
config.args.__setattr__('phi', id_)
config.args.__setattr__('pi', multi)
config.args.__setattr__('k', 2)
g1, g2 = gradient_similarity(config.args, ce_loss, trunc_ce, gumbel)

alpha:  1.0
Cosine Similarity between grad 1 and grad 2:  tensor([0.9990, 0.9779])


Compare CE Loss with Truncated CE Loss with Logit Ball Oracle

In [12]:
config.args.__setattr__('pi', multi)
config.args.__setattr__('k', 2)
config.args.__setattr__('phi', oracle.DNN_Logit_Ball(ch.full(ch.Size([config.args.K,]), -2, dtype=ch.float32), ch.full(ch.Size([config.args.K,]), 2, dtype=ch.float32)) )
g1, g2 = gradient_similarity(config.args, ce_loss, trunc_ce, gumbel)

alpha:  0.4943
Cosine Similarity between grad 1 and grad 2:  tensor([0.7911, 0.6006])


Compare BCE Loss with Truncated BCE Loss and Idenity Oracle

In [20]:
config.args.__setattr__('phi', id_)
config.args.__setattr__('pi', binary)
config.args.__setattr__('k', 1)
g1, g2 = gradient_similarity(config.args, bce_logit, trunc_bce, logistic)

alpha:  1.0
Cosine Similarity between grad 1 and grad 2:  tensor([0.9999])


Compare BCE Loss with Truncated BCE Loss with Ball Oracle

In [21]:
config.args.__setattr__('pi', binary)
config.args.__setattr__('k', 1)
config.args.__setattr__('phi', oracle.DNN_Logit_Ball(ch.full(ch.Size([config.args.K,]), -2, dtype=ch.float32), ch.full(ch.Size([config.args.K,]), 2, dtype=ch.float32)) )
g1, g2 = gradient_similarity(config.args, bce_logit, trunc_bce, logistic)

alpha:  0.5456
Cosine Similarity between grad 1 and grad 2:  tensor([0.9828])


Compare the Diff in the CE Loss with BCE Loss 

In [29]:
config.args.__setattr__('pi', binary)
config.args.__setattr__('k', 1)
config.args.__setattr__('phi', oracle.DNN_Logit_Ball(ch.full(ch.Size([config.args.K,]), -2, dtype=ch.float32), ch.full(ch.Size([config.args.K,]), 2, dtype=ch.float32)) )
g1, g2 = gradient_similarity_gumbel_diff(config.args, gumbel_ce, logistic_bce, logistic)

alpha:  0.3857


IndexError: Target 1 is out of bounds.

In [74]:
config.args.__setattr__('pi', multi)
config.args.__setattr__('k', 2)
config.args.__setattr__('phi', id_)
g1, g2 = gradient_similarity(config.args, ce_loss, gumbel_bce, gumbel)

print("cosine diff:", ch.nn.CosineSimilarity(dim=0)(g1[:,0] - g1[:,1], g2[:,0]))

alpha:  1.0
Cosine Similarity between grad 1 and grad 2:  tensor([0.9985, 0.0000])
cosine diff: tensor(0.9985)


In [13]:
class TruncatedBCE_(ch.autograd.Function):
    @staticmethod
    def forward(ctx, pred, targ):
        ctx.save_for_backward(pred, targ)
        loss = ch.nn.BCEWithLogitsLoss()
        return loss(pred, targ)

    @staticmethod
    def backward(ctx, grad_output):
        pred, targ = ctx.saved_tensors

        # logistic distribution
        base_distribution = Uniform(0, 1)
        transforms_ = [SigmoidTransform().inv]
        logistic = TransformedDistribution(base_distribution, transforms_)

        stacked = pred[None, ...].repeat(config.args.num_samples, 1, 1)
        rand_noise = logistic.sample(stacked.size())
        # add noise
        noised = stacked + rand_noise
        noised_labs = noised > 0
        filtered = config.args.phi(noised)
        # filter
        mask = (noised_labs).eq(targ) * filtered
        avg = 2 * (((sig(rand_noise)*mask).sum(0) / (mask.sum(0) + 1e-5)) - (sig(rand_noise)*filtered).sum(0) / (filtered.sum(0) + 1e-5))
        return -avg, None
    
trunc_bce_ = TruncatedBCE_.apply

In [15]:
config.args.__setattr__('phi', ball)
config.args.__setattr__('pi', binary)
config.args.__setattr__('k', 1)
g1, g2 = gradient_similarity(config.args, bce_logit, trunc_bce_, logistic)

NameError: name 'ball' is not defined

In [85]:
config.args.__setattr__('pi', binary)
config.args.__setattr__('k', 1)
config.args.__setattr__('phi', oracle.DNN_Lower(-.1))
g1, g2 = gradient_similarity(config.args, bce_logit, trunc_bce, logistic)

alpha:  0.5264
Cosine Similarity between grad 1 and grad 2:  tensor([0.8523])
